In [1]:
import sqlite3
import pandas as pd
import csv
import numpy as np

In [11]:
conn = sqlite3.connect('fin_db2.db')
cursor = conn.cursor()


In [12]:
cursor.execute('''CREATE TABLE IF NOT EXISTS fin_db2 (
                    id INTEGER,
                    text TEXT,
                    embeding INTEGER
                )''')

In [13]:
df = pd.read_excel('../../igor_bd.xlsx', names = ['Sentences', 'Embbeding'])

In [14]:
df['index'] = df.index + 1

In [15]:
df[['index','Embbeding', 'Sentences']].to_csv('out1.csv', index = False)

In [16]:
with open('out1.csv', 'r') as file:
    csv_data = csv.reader(file)
    next(csv_data)  # Пропуск заголовков столбцов
    cursor.executemany('INSERT INTO fin_db2 VALUES (?, ?, ?)', csv_data)

In [17]:
pd.read_csv('out1.csv')

,index,Embbeding,Sentences
0,1,"0.05866908,-0.055919643,0.028513093,0.00883587...",Область применения Настоящая Политика определя...
1,2,"0.04113537,-0.09494297,-0.020927062,0.03347726...","Физические лица, с которыми заключен договор г..."
2,3,"0.0520625,-0.031804126,-0.0013829818,0.0305636...",Настоящая Политика обязательна к ознакомлению ...
3,4,"0.0671922,-0.028048074,-0.0023852815,0.0003226...","Недопущение и предупреждение ситуаций, когда д..."
4,5,"0.03956517,-0.06380333,0.022312013,0.026933933...",Определение обязательных для соблюдения Банком...
...,...,...,...
208,209,"0.0012677034,-0.08318724,-0.07216746,0.0066744...",Если имеющихся у Банка денежных средств недост...
209,210,"0.019662531,-0.099769615,-0.07005769,0.0257089...",После завершения расчетов с кредиторами ликвид...
210,211,"0.077003926,-0.079033636,-0.068083644,0.023692...","18.5. Ликвидация считается завершенной, а Банк..."
211,212,"0.029698445,-0.05572576,-0.061276782,0.0276058...",Передача документов осуществляется за счет Бан...


In [18]:
conn.commit()
conn.close()

In [19]:
conn = sqlite3.connect('fin_db2.db')
cursor = conn.cursor()

In [23]:
cursor.execute("SELECT * FROM fin_db2")
rows = cursor.fetchall()

In [26]:
rows[0][2]

'Область применения Настоящая Политика определяет основные цели и задачи управления конфликтом интересов в Банке, ключевые принципы и меры по минимизации риска конфликта интересов (в том числе порядок предотвращения конфликтов интересов) и соблюдения норм применимого законодательства. Политика является частью системы управления комплаенс-риском, создана для эффективного управления риском возникновения конфликта интересов в Банке и обязательна к выполнению всеми Работниками Банка.'

In [39]:
input_vector = [float(x) for x in rows[0][1].split(',')]

In [40]:
# vectors = [np.frombuffer(vector[0]) for vector in vectors]

In [41]:
from scipy.spatial.distance import cosine

In [50]:
# Расчет косинусного расстояния и сохранение результатов
distances = []
for row in rows:
    vector = [float(x) for x in row[1].split(',')]
    distance = cosine(input_vector, vector)
    distances.append((distance, row[1], row[0],row[2]))  # Замените на соответствующий столбец с другими данными

# Сортировка результатов по возрастанию расстояния
distances.sort(key=lambda x: x[0])

# Вывод 10 самых близких расстояний и соответствующих данных
# for i in range(10):
#     print(f"Расстояние: {distances[i][0]}, Другие данные: {distances[i][1]}")

# Закрытие соединения с базой данных
conn.close()

In [57]:
number_sent = 2
out = []
ids = []
for i in range(number_sent):
    out.append(distances[i][3])
    ids.append(distances[i][2])

In [58]:
ids

[1, 49]

In [44]:
out = []
for i in range(10):
    out.append(distances[i][2])

In [45]:
out

[1, 49, 4, 26, 39, 22, 29, 50, 48, 51]

In [ ]:
После создания базы данных, вы можете использовать ее внутри Docker контейнера. Для этого вам нужно создать Dockerfile, который будет содержать инструкции для создания контейнера. Вот пример Dockerfile:

```Dockerfile
FROM python:3.9

# Копирование файлов в контейнер
COPY database.db /app/database.db
COPY script.py /app/script.py
COPY data.csv /app/data.csv

# Установка зависимостей
RUN pip install sqlite3

# Запуск скрипта при запуске контейнера
CMD python /app/script.py
```

Затем, вы можете собрать Docker образ и запустить контейнер с помощью следующих команд:

```
docker build -t my_app .
docker run my_app
```

При этом, файлы `database.db`, `script.py` и `data.csv` должны находиться в той же директории, где находится Dockerfile.

In [64]:
s = 'erfghjkjhgfde.asdfghmjretwgfhertw.ewqrtfgnmhrtewgfhret3452regfhrt435.q4werdghfertwrgfdert342ew.rsdfsdfsdf'
n = 10
a = []
cnt = 0
cur = ''
for i in s:
    cur += i
    if cnt>n and i == '.':
        a.append(cur)
        cur = ''
        cnt = 0
    else:
        cnt+=1
    

In [65]:
a

['erfghjkjhgfde.',
 'asdfghmjretwgfhertw.',
 'ewqrtfgnmhrtewgfhret3452regfhrt435.',
 'q4werdghfertwrgfdert342ew.']